## This notebook illustrates how simulated difference PED patterns are compared with experimental diff of VSAD images

In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from skimage.transform import resize, rotate, downscale_local_mean, resize_local_mean
from skimage.filters import gaussian
from scipy.signal import correlate2d
import pyxem as pxy
plt.rcParams["svg.fonttype"]  = "none"
plt.rcParams["axes.unicode_minus"] = False

In [3]:

dp_exp_1deg_10ms = hs.load("/Users/anders/Library/CloudStorage/OneDrive-NTNU/Master/Data/2100F/22_03_23/20230322_181923/SPED_128_128_spot1_alph5_CL12_tx2p7_ty2p0_1deg_step13p9nm/SPED_128_128_spot1_alph5_CL12_tx2p7_ty2p0_1deg_step13p9nm.hspy")


diff_vsad_sim_1 = hs.load("/Users/anders/Library/CloudStorage/OneDrive-NTNU/Master/Data/py_multislice/1deg/SPEDsim_1deg_c_up_minus_down.hspy")


In [5]:
roi1 = hs.roi.CircleROI(cx=514.3, cy=986.9, r=118.15, r_inner=0)
roi2 = hs.roi.CircleROI(cx=625.5, cy=1278.8, r=118.15, r_inner=0)

vsad1 = roi1(dp_exp_1deg_10ms).nansum()
vsad2 = roi2(dp_exp_1deg_10ms).nansum()


diff_vsad_exp = np.zeros(vsad1.data.shape)
diff_vsad_exp = (vsad1.data - vsad2.data)

diff_vsad_exp = rotate(diff_vsad_exp, -7.5)

diff_vsad_exp = hs.signals.Signal2D(diff_vsad_exp)

exp_crop_rect = hs.roi.RectangularROI(left=41, top=46, right=214, bottom=209)
diff_vsad_exp.plot()
exp_crop_rect.add_widget(diff_vsad_exp)

In [6]:
exp_cropped = exp_crop_rect(diff_vsad_exp)

In [7]:
#One deg

diff_vsad_sim_1.plot()
sim_crop_rect = hs.roi.RectangularROI(left=404, top=404, right=961, bottom=961)

sim_crop_rect.add_widget(diff_vsad_sim_1)

In [8]:
#One deg

sim_cropped = sim_crop_rect(diff_vsad_sim_1)

In [9]:
cx, cy = exp_cropped.data.shape
print(sim_cropped.data.shape)

(300, 557, 557)


In [10]:
sim_cropped.data = resize_local_mean(sim_cropped.data,(300,)+ exp_cropped.data.shape)

sim_cropped = hs.signals.Signal2D(sim_cropped.data)

In [11]:
dx = 10
dy = 10

sim_cropped.data[:, cx//2 - dx:cx//2+ dx,cy//2 -dy:cy//2 + dy]=0
exp_cropped.data[cx//2 - dx:cx//2+ dx,cy//2 -dy:cy//2 + dy] =0

In [12]:
exp_cropped.data = gaussian(exp_cropped.data)
sim_cropped.data = gaussian(sim_cropped.data)

In [13]:

def norm_data(data):
    """
    normalize data to have mean=0 and standard_deviation=1
    """
    mean_data=np.mean(data)
    std_data=np.std(data, ddof=1)
    #return (data-mean_data)/(std_data*np.sqrt(data.size-1))
    return (data-mean_data)/(std_data)


def ncc(data0, data1):
    """
    normalized cross-correlation coefficient between two data sets

    Parameters
    ----------
    data0, data1 :  numpy arrays of same size
    """
    return (1.0/(data0.size-1)) * np.sum(norm_data(data0)*norm_data(data1))


In [19]:
thick = np.linspace(250,1500,300)
corrs = []
corrs_inv = []

for i in range(sim_cropped.data.shape[0]):
    corrs.append(ncc(sim_cropped.data[i].ravel(), exp_cropped.data.ravel()))
    corrs_inv.append(ncc(sim_cropped.data[i,::-1,::-1].ravel(), exp_cropped.data.ravel()))

corrs = np.array(corrs)
corrs = corrs.squeeze()

plt.figure()
plt.plot(thick /10 , corrs)
plt.plot(thick /10 , corrs_inv, label  = "Down-Up")
print("Argmax", np.argmax(corrs))
print(f"Thickness = {thick[np.argmax(corrs)] :.2f} Å")
plt.xlabel("Thickness [nm]")
plt.ylabel("NCC Score")
plt.title("Down - Up")
plt.grid()


Argmax 110
Thickness = 709.87 Å


In [20]:
sim_cropped.axes_manager[0].index = np.argmax(corrs)

sim_cropped.plot(cmap="RdBu")
exp_cropped.plot(cmap="RdBu")